In [2]:
import os
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import sys
import numpy as np

img_name = None
submit = None

def open_image():
    global img_name
    img_name = filedialog.askopenfilename(
        initialdir=".",
        title="Select Image",
        filetypes=(("Images", "*.jpg;*.jpeg;*.png;*.bmp"), ("All Files", "*.*"))
    )
    process_image()

def process_image():
    global submit
    if not img_name:
        return

    img = Image.open(img_name)
    img = img.resize((250, 250), Image.LANCZOS)

    l1 = tk.Label(root, text="Original Image:")
    l1.grid(column=0, row=2)

    img = ImageTk.PhotoImage(img)
    l2 = tk.Label(root, image=img)
    l2.image = img
    l2.grid(column=0, row=3)

    submit = tk.Button(root, text="Submit", command=remove_haze)
    submit.grid(column=0, row=4)

def remove_haze():
    global img_name, submit

    if not img_name:
        return

    submit.destroy()

    # Load the selected image
    img = cv2.imread(img_name)

    # Perform dehazing using the dark channel prior method
    dark_channel = np.minimum.reduce([img[:, :, 0], img[:, :, 1], img[:, :, 2]])
    atmospheric_light = np.percentile(dark_channel, 95)
    transmission = 1 - 0.7 * (dark_channel / atmospheric_light)
    transmission = np.maximum(transmission, 0.1)  # Ensure minimum transmission value

    dehazed_image = np.zeros_like(img)
    for channel in range(3):
        dehazed_image[:, :, channel] = (img[:, :, channel] - atmospheric_light) / transmission + atmospheric_light

    dehazed_image = np.uint8(np.clip(dehazed_image, 0, 255))

    # Convert BGR to RGB
    dehazed_image = cv2.cvtColor(dehazed_image, cv2.COLOR_BGR2RGB)

    # Display dehazed image
    img = Image.fromarray(dehazed_image)
    img = img.resize((250, 250), Image.LANCZOS)

    l3 = tk.Label(root, text="Dehazed Image:")
    l3.grid(column=1, row=2)

    img = ImageTk.PhotoImage(img)
    l4 = tk.Label(root, image=img)
    l4.image = img
    l4.grid(column=1, row=3, padx=10)

    retry = tk.Button(root, text="Retry", command=restart_program)
    retry.grid(column=0, row=5)

    quit_button = tk.Button(root, text="Quit", command=quit_program)
    quit_button.grid(column=1, row=5)

    root.update_idletasks()


def restart_program():
    os.execl(sys.executable, sys.executable, *sys.argv)

def quit_program():
    sys.exit()

root = tk.Tk()
root.title("Dehaze")
root.update_idletasks()

label = tk.Label(root, text="Select an image to remove haze:")
label.grid(column=0, row=0)

input = tk.Entry(root, width=50)
input.grid(column=0, row=1, padx=10, pady=10)

browse = tk.Button(root, text="Browse", command=open_image)
browse.grid(column=1, row=1)

root.mainloop()